# Example Notebook

This notebook serves to show how to create a notebook in the current directory structure.

Simply, after importing `init_notebook` the user can both import the `test_module` and its `test_function`, located in the `src` directory. Finally, it also becomes possible to open files in the `data` folder directly. All paths are relative to this directory so it is also possible to directly save a file to it.

In [ ]:
import numpy as np
# matplotlib pyplot
import matplotlib.pyplot as plt

import init_notebook
import task1_utils as utils

In [ ]:
global linear_data, nonlinear_data
linear_data = np.loadtxt("task_1/linear_function_data.txt", delimiter=" ")
nonlinear_data = np.loadtxt("task_1/nonlinear_function_data.txt", delimiter=" ")
assert linear_data.shape == (1000, 2) and nonlinear_data.shape == (1000, 2)

# Visualize data

We want to create an approximation function $\hat{f}: \mathbb{R} \to \mathbb{R}$

In [ ]:
plt.scatter(linear_data[:, 0], linear_data[:, 1])
plt.scatter(nonlinear_data[:, 0], nonlinear_data[:, 1])
plt.legend(["linear", "nonlinear"])
plt.show()

# Solving linear data

Since we ignore linear-affine cases, The linear data is created by a function $$f: \mathbb{R} \to \mathbb{R}$$ $$x \mapsto \alpha x$$

We use **least square minimization** to find the best approximation function $\hat{f}$, $\hat{f}(x) = \beta x$ given our data, where our error is given by the squares of the differences between $\hat{f}$ and $f$.

We can formulate our problem as a vectorized linear equation:

$$\begin{bmatrix}
x_1 \\
x_2  \\
\vdots \\
x_{1000}
\end{bmatrix}
\beta =
\begin{bmatrix}
y_1 \\
y_2 \\
\vdots \\
y_{1000}
\end{bmatrix}$$

The error function is given by the squared mean of the residuals:

$$\min_{\hat{f}}e(\hat{f}) \propto \min_{\beta} \sum_{i=1}^{1000} (\beta x_i - y_i)^2$$

The solution for $\beta$ given by the least square minimization minimizes our error function. In the `numpy.linalg.lstsq` function, the solution $\beta$ is given by a 1x1 matrix.

In [ ]:
solution_linear, residuals = utils.linear_approximate_1d(linear_data[:, 0], linear_data[:, 1])
solution_closed_form = utils.linear_approximate_closed_form(linear_data[:, 0], linear_data[:, 1])
assert np.allclose(solution_linear, solution_closed_form)

print(f'Solution: {solution_linear}, residual: {residuals}')


# also plot nonlinear fit to linear data
l_vector = np.linspace(np.min(linear_data[:, 0]), np.max(linear_data[:,0]), 10)
utils.rbf_interpolate(linear_data[:, 0], linear_data[:, 1], 0.75, l_vector)

# Approximate nonlinear data with a linear function

Same procedure as above but the results are poor:

Solution: 0.03321036, residual: 774.8919879

In [ ]:
solution_linear, residuals = utils.linear_approximate_1d(nonlinear_data[:, 0], nonlinear_data[:, 1])
solution_closed_form = utils.linear_approximate_closed_form(nonlinear_data[:, 0], nonlinear_data[:, 1])
assert np.allclose(solution_linear, solution_closed_form)

# Approximate nonlinear data with radial basis functions

We notice that the data is not linear, but rather a sinusoidal function. We can approximate this function with a linear combination of radial basis functions. We count the peaks and troughs of the data and use this as the number of radial basis functions. This is because our basis of nonlinear functions is given by the following:

$$\phi_l(x) = \exp\left(-\left(\frac{x-x_l}{\epsilon^2}\right)^2\,\,\right)$$

This denotes a sharp peak around $x_l$. In order to mimic our data, we use a linear combination of $\phi_l$ functions.

Our apoproximation function is given by a linear combination of these functions: $$\hat{f}(x) = \sum_{l=1}^L c_l\, \phi_l(x)$$

The only unknowns are the coefficients $c_l$ and the width $\epsilon$ of the peaks. We can formulate our problem as a linear equation:

$$\begin{bmatrix}
\phi_1(x_1) & \phi_2(x_1) & \cdots & \phi_L(x_1) \\
\phi_1(x_2) & \phi_2(x_2) & \cdots & \phi_L(x_2) \\
\vdots & \vdots & \ddots & \vdots \\
\phi_1(x_{1000}) & \phi_2(x_{1000}) & \cdots & \phi_L(x_{1000})
\end{bmatrix}
\begin{bmatrix}
c_1 \\
c_2 \\
\vdots \\
c_L
\end{bmatrix} =
\begin{bmatrix}
y_1 \\
y_2 \\
\vdots \\
y_{1000}
\end{bmatrix}$$

We use least square minimization to find the best coefficients $c_l$.

We need $L \ge 6$, otherwise we cannot have enough peaks and troughs.

In [ ]:
max_x = nonlinear_data[np.argmax(nonlinear_data[:, 1]), 0]
max_y = nonlinear_data[np.argmax(nonlinear_data[:, 1]), 1]


plt.scatter(nonlinear_data[:, 0], nonlinear_data[:, 1])
x = np.linspace(max_x-1, max_x+1, 1000)
# Test different values of epsilon to see what shape fits best. Focus on the shape around the peak.
for epsilon in [0.3, 0.5, 0.6]:
  # scale by max_y to fit the peak itself
  y = max_y * np.exp(-((x-max_x)/epsilon)**2)
  plt.plot(x, y, color = "red", linewidth=1)
plt.title(f'Radial axis around peak with different values of epsilon')
# restrict x axis to -1 and 2
plt.xlim(-1, 2)
plt.show()

# 1. Even spacing of $x_l$ values

In [ ]:
# Change epsilon to your liking here
epsilon = 2
min = np.min(nonlinear_data[:, 0])
max = np.max(nonlinear_data[:, 0])

l_vector = np.linspace(min, max, 8)
utils.rbf_interpolate(nonlinear_data[:, 0], nonlinear_data[:, 1], 2, l_vector)

l_vector = np.linspace(min, max, 10)
utils.rbf_interpolate(nonlinear_data[:, 0], nonlinear_data[:, 1], 2, l_vector)

# Cherrypicked $x_l$ values

This has delivered the best results. Unfortunately, this is not a general solution and is difficult to automate, especially for more complex or higher-dimensional data.

In [ ]:
# cherrypicked vector
l_vector = np.array([ -2.4, -1.35, -0.70, 0.4, 1.6, 2.7])

utils.rbf_interpolate(nonlinear_data[:, 0], nonlinear_data[:, 1], 0.7, l_vector)

l_vector = np.array([min, -2.4, -1.35, -0.70, 0.4, 1.6, 2.7, max])
utils.rbf_interpolate(nonlinear_data[:, 0], nonlinear_data[:, 1], 0.7, l_vector)

In [ ]:
l_vector = utils.get_extrema(nonlinear_data[:, 0], nonlinear_data[:, 1], 15)
utils.rbf_interpolate(nonlinear_data[:, 0], nonlinear_data[:, 1], epsilon, l_vector)

l_vector = utils.get_extrema(nonlinear_data[:, 0], nonlinear_data[:, 1], 10)
utils.rbf_interpolate(nonlinear_data[:, 0], nonlinear_data[:, 1], epsilon, l_vector)